In [ ]:
%pip install openai langchain requests bs4 langchain-openai dotenv pandas


ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
Note: you may need to restart the kernel to use updated packages.


In [4]:
import requests
import json

url = "https://api.docsbot.ai/teams/wA9muL1YoBvL3tk9qicM/bots/8Hjbk7AwcRataBBe8cBI/search"

payload = json.dumps({
  "query": "hur påverkar litiumbehandling kreatininvärde?",
  "top_k": 10,
  "alpha": 0.75
})

headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

[{"title":"Intoxikation – Litium","url":"https://www.internetmedicin.se/akutsjukvard/intoxikation-litium","page":null,"content":"# Intoxikation – Litium\n\n### Författare:\n\n*   [Bengt R. Widgren](https://www.internetmedicin.se/people/bengt-r-widgren)\n\n    (Docent. Predicare AB / Göteborg)\n\n### Reviderad:\n\n#### 2025-04-15\n\nBehandling av förgiftning orsakad av litium.\n\n* * *\n\nLitiumcitrat (se [litium](https://www.fass.se/LIF/result?userType=0&query=litium)) används framför allt vid behandling av affektiva syndrom, t ex manodepressiv sjukdom. Normal serumkoncentration vid behandling är < 1,2 mmol/l 12 timmar efter tablettintag.\n\nLitium har en snäv terapeutisk bredd, vilket gör det svårinställt framför allt hos äldre och patienter med nedsatt njurfunktion. Vissa preparat ökar litiums toxicitet vid terapeutiska nivåer, t ex:\n\n*   Dehydrering\n\n*   Antihypertensiva läkemedel\n\n*   Metronidazol\n\n*   NSAIDs\n\n*   Diuretika\n\n*   Tetracykliner\n\n*   Fenytoin\n\nVid förg

In [10]:
import requests
from bs4 import BeautifulSoup
import time

def scrape_internetmedicin(url):
    # 1. Define Headers
    # Internetmedicin will likely block you if you don't look like a real browser.
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # 2. Fetch the content
    print(f"Fetching {url}...")
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"Failed to retrieve page. Status code: {response.status_code}")
        return

    # 3. Parse with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # 4. Extract Data
    # Note: You must inspect the website to find the correct Class Name. 
    # At the time of writing, Internetmedicin often uses 'article-body' or generic headers.
    
    # Let's try to find the main article title
    title = soup.find('h1')
    print(f"\nTitle: {title.get_text(strip=True) if title else 'No title found'}")

    print("-" * 30)

    # Let's try to extract all headings and paragraphs
    # This finds the main content area (Note: Class names like 'c-article' might change!)
    # A safe bet is often to just look for all <p> tags if you want raw text.
    
    paragraphs = soup.find_all('p')
    
    print(f"Found {len(paragraphs)} paragraphs. Showing the first 50:\n")
    for p in paragraphs[:50]:
        print(p.get_text(strip=True))

# Run the function
url_im = 'https://www.internetmedicin.se/kardiologi/hjartamyloidos-amyloid-kardiomyopati'
scrape_internetmedicin(url_im)

Fetching https://www.internetmedicin.se/kardiologi/hjartamyloidos-amyloid-kardiomyopati...

Title: Hjärtamyloidos (amyloid kardiomyopati)
------------------------------
Found 125 paragraphs. Showing the first 50:

(Specialistläkare.Centrum för kardiovaskulär genetik /Norrlands universitetssjukhus)
(Seniorprofessor.Medicinkliniken /SU)
Hjärtamyloidos klassificeras som en hypertrofisk och restriktiv kardiomyopati, d v s ger en hjärtmuskelförtjockning och nedsatt eftergivlighet i myokardiet.
Annons1av3
Förkortad produktinformation
Veltassa(patiromer), 8,4 g och 16,8 g pulver till oral suspension, Rx, (F), Farmakoterapeutisk grupp: V03AE09, läkemedel för behandling av hyperkalemi och hyperfosfatemi.Indikation:Behandling av hyperkalemi hos vuxna och ungdomar i åldern 12–17 år.Kontraindikation:Överkänslighet mot den aktiva substansen eller mot något hjälpämne.Varning och försiktighet:Veltassa ska inte användas som ersättning för akut behandling av livshotande hyperkalemi. Serummagnesium bör 

In [7]:
def scrape_fass(npl_id):
    # URL structure for FASS professionals (Vårdpersonal)
    base_url = f"https://www.fass.se/LIF/product?userType=0&nplId={npl_id}"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Referer': 'https://www.fass.se/',
        'Accept-Language': 'sv-SE,sv;q=0.9,en-US;q=0.8,en;q=0.7'
    }

    print(f"\nFetching FASS data for ID: {npl_id}...")
    response = requests.get(base_url, headers=headers)

    if response.status_code != 200:
        print("Failed to connect to FASS.")
        return

    soup = BeautifulSoup(response.content, 'html.parser')

    # FASS content is often inside a div with id="content" or specific article tags.
    # A common element for the drug name is an <h1> with class "product-name"
    
    product_name = soup.find('h1', class_='product-name')
    if product_name:
        print(f"Drug Name: {product_name.get_text(strip=True)}")
    
    # Try to find the "Indikationer" (Indications) section.
    # FASS often uses <h3> headers for sections. We can search for the text "Indikationer".
    
    indication_header = soup.find('h3', string=lambda t: t and "Indikationer" in t)
    
    if indication_header:
        print("\n--- Indikationer ---")
        # The text is usually in the sibling div or the next paragraph
        content = indication_header.find_next_sibling()
        if content:
            print(content.get_text(strip=True))
    else:
        print("Could not find 'Indikationer' section (content might be loaded via JS).")

# Vyndaqel 61 mg NplId (Example ID, you must find the correct one in the URL)
# Example ID for Vyndaqel 61 mg: 20180216000078 
scrape_fass("20180216000078")


Fetching FASS data for ID: 20180216000078...
Failed to connect to FASS.


In [8]:
import requests
from bs4 import BeautifulSoup
import time

def scrape_internetmedicin(url):
    # 1. Define Headers
    # Internetmedicin will likely block you if you don't look like a real browser.
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # 2. Fetch the content
    print(f"Fetching {url}...")
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"Failed to retrieve page. Status code: {response.status_code}")
        return

    # 3. Parse with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # 4. Extract Data
    # Note: You must inspect the website to find the correct Class Name. 
    # At the time of writing, Internetmedicin often uses 'article-body' or generic headers.
    
    # Let's try to find the main article title
    title = soup.find('h1')
    print(f"\nTitle: {title.get_text(strip=True) if title else 'No title found'}")

    print("-" * 30)

    # Let's try to extract all headings and paragraphs
    # This finds the main content area (Note: Class names like 'c-article' might change!)
    # A safe bet is often to just look for all <p> tags if you want raw text.
    
    paragraphs = soup.find_all('p')
    
    print(f"Found {len(paragraphs)} paragraphs. Showing the first 3:\n")
    for p in paragraphs[:3]:
        print(p.get_text(strip=True))
        print("...")

# Run the function
url_im = 'https://fass.se/health/product/20240110000048/smpc'
scrape_internetmedicin(url_im)

Fetching https://fass.se/health/product/20240110000048/smpc...

Title: BEYONTTRA, Filmdragerad tablett 356 mg
------------------------------
Found 592 paragraphs. Showing the first 3:

Säkerhetsmeddelande till marknaden2026-01-05, läs mer här.
...
Vita, ovala filmdragerade tabletter cirka 15 mm × 7,5 mm, med ”BEYONTTRA” tryckt i svart bläck på ena sidan.
...
Vita, ovala filmdragerade tabletter cirka 15 mm × 7,5 mm, med ”BEYONTTRA” tryckt i svart bläck på ena sidan.
...


In [9]:
import requests
from bs4 import BeautifulSoup

def scrape_clean_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 1. Rensa bort tekniska taggar som ofta stör eller innehåller dolda annonser
    # 'script' och 'style' är standard att ta bort. 
    # 'iframe' används ofta för att ladda in tunga annonser.
    unwanted_tags = ['script', 'style', 'iframe', 'noscript']
    
    for tag in soup.find_all(unwanted_tags):
        tag.decompose()  # Tar bort taggen helt från trädet

    # 2. Rensa bort specifika annonsklasser (Exempel - du måste kolla vad sidan använder!)
    # På svenska sidor kan de heta saker som "annons", "reklam", "banner-top" osv.
    ad_classes = ['ad-container', 'banner-wrapper', 'cookie-notice', 'marketing-box', 'annons']
    
    # Vi letar efter div:ar som har någon av dessa klasser
    for ad_class in ad_classes:
        # Hitta alla div:ar med denna klass (om de finns)
        ads = soup.find_all('div', class_=ad_class) 
        for ad in ads:
            ad.decompose()

    # 3. Nu när "soppan" är ren, hämta texten
    # get_text() med separator=' ' ser till att ord inte klistras ihop
    clean_text = soup.get_text(separator=' ', strip=True)
    
    return clean_text

# Exempelanvändning
print(scrape_clean_text('https://fass.se/health/product/20240110000048/smpc'))

BEYONTTRA (Filmdragerad tablett 356 mg) • Produktresumé | FASS Vård Tillgänglig läkemedelsinformation i Sverige Kontakta oss FAQ Fass Vård Fass Patient Vård Djur MENY / Läkemedel / BEYONTTRA, Filmdragerad tablett 356 mg / Produktresumé Vita, ovala filmdragerade tabletter cirka 15 mm × 7,5 mm, med ”BEYONTTRA” tryckt i svart bläck på ena sidan. Vita, ovala filmdragerade tabletter cirka 15 mm × 7,5 mm, med ”BEYONTTRA” tryckt i svart bläck på ena sidan. Fler bilder | Info om att dela och krossa Bayer AB BEYONTTRA, Filmdragerad tablett 356 mg Medel vid hjärtsjukdomar, övriga medel vid hjärtsjukdomar Vita, ovala filmdragerade tabletter cirka 15 mm × 7,5 mm, med ”BEYONTTRA” tryckt i svart bläck på ena sidan. Vita, ovala filmdragerade tabletter cirka 15 mm × 7,5 mm, med ”BEYONTTRA” tryckt i svart bläck på ena sidan. Fler bilder | Info om att dela och krossa Om läkemedlet Utökad övervakning Recept krävs Ingår i högkostnadsskyddet med begränsning Läkemedel från Bayer AB omfattas av Läkemedelsför

In [8]:
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from dotenv import load_dotenv
import os

load_dotenv()
api_key=os.environ["JOHAN_AZURE_OPENAI_KEY"]
api_version="2024-12-01-preview"
api_endpoint = os.environ["JOHAN_AZURE_OPENAI_ENDPOINT"]
deployment= "gpt-5-chat"

llm = AzureChatOpenAI(
    azure_deployment= deployment,
    api_version=api_version,
    azure_endpoint=api_endpoint,
    api_key=api_key,
    temperature=0.0
)

/Users/johanwiding/miniconda3/envs/internetmedicin/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [19]:
def create_quiz(article, läkemedel, questions=5):

    prompt_template = """
    -Roll-
    Du är en expert på att skapa utbildningsmaterial och provfrågor av hög kvalitet (t.ex. för läkare eller studenter).

    -Mål-
    Din uppgift är att läsa den bifogade artikeln och skapa ett quiz bestående av {questions} frågor för att testa läsarens förståelse.
    Ta fram frågor som berör olika behandling av amyloid kardiomyopati. Fokusera främst på den allmäna behandlingen och gör 1-2 frågor om specifika läkemedel.

    -Instruktioner-
    1. Analysera artikeln noggrant.
    3. Skapa {questions} flervalsfrågor.
    4. Varje fråga ska ha 4 svarsalternativ (A, B, C, D) där endast ett är korrekt.
    5. **Frågeformat:** Frågestammen får gärna vara utförlig och beskriva ett scenario, en klinisk fallstudie eller en kontext.
    6. **Svarsalternativ:** Dessa ska vara koncisa. De felaktiga svaren (distraktorerna) måste vara rimliga och troliga för en okunnig person, men entydigt felaktiga baserat på texten.
    7. **Motivering:** Inkludera en utförlig pedagogisk förklaring till varför rätt svar är korrekt.

    -Indata-
    Artikel:
    {article}
    Information om specifika läkemedel:
    {läkemedel}

    Inte för många frågor får handla om specifika läkemedel. Fokusera på allmän behandling.
    """
    """-Utdata-
    Returnera ENDAST rå JSON (ingen markdown-formatering eller övrig text) enligt följande struktur:
    {{
      "quiz": [
        {{
          "question": "Här skriver du frågan/scenariot...",
          "options": {{
            "A": "Alternativ A",
            "B": "Alternativ B",
            "C": "Alternativ C",
            "D": "Alternativ D"
          }},
          "correct_answer": "A",
          "motivation": "Förklaring till varför A är rätt baserat på texten."
        }}
      ]
    }}
    """
    prompt = ChatPromptTemplate.from_template(prompt_template)
    inputs = {
        "article": article,
        "questions": questions,
        "läkemedel": läkemedel

    }
    chain = prompt | llm | StrOutputParser()
    response = chain.invoke(inputs)
    return response

In [21]:
article = """

Amyloidos är ett begrepp som används för en heterogen samling av sjukdomar och\ninnebär någon form av proteininlagring i ett eller flera organ. De har alla\ngemensamt att olika mekanismer leder fram till felveckade proteinstrukturer\nvilka är svårnedbrytbara. Upp mot 25 olika proteintyper som kan ge upphov till\nsystemisk amyloidos har beskrivits hos människan.\n\nDen defekta proteinstrukturen vid AL-amyloidos utgörs av lätta\nimmunglobulinkedjor (AL står således för amyloidosis light chain) producerade av\nklonala plasmaceller. Det är 2-3 gånger vanligare att den lätta kedjan vid\nAL-amyloidos är av lambdatyp än av kappa. AL amyloidos är en ovanlig sjukdom med\nen årlig incidens på ca 1/100 000. Medianåldern vid diagnos är ca 70 år och ses\nnågot oftare hos män än kvinnor.\n\n\nETIOLOGI\n\nPatienter med AL-amyloidos har en klonal plasmacellsjukdom, oftast MGUS\n(monoclonal gammopathy of unknown significance) eller mera sällan myelom. Det\nfinns idag inga kända riskfaktorer för att utveckla AL-amyloidos. Sjukdomen är\ninte ärftlig.\n\nEn stor svårighet vid diagnostik av AL-amyloidos är att den kliniska bilden kan\nvariera mycket beroende på vilket/vilka organsystem som är påverkade. Sjukdomen\nbör övervägas i alla fall med oklar genes vid:\n\n * Proteinuri, framförallt albuminuri\n\n * Kardiomyopati (främst restriktiv)\n\n * Sensorisk polyneuropati\n\n * Hepatomegali\n\nSymtom som har betraktats som närmast patognomona är periorbitala blödningar och\nmakroglossi men detta förekommer bara i en minoritet av fallen.\n\nEftersom flera olika sjukdomar kan leda fram till proteininlagring är det av\nstörsta vikt att korrekt subtypning av amyloidosen sker. Vid misstanke om\nAL-amyloidos skall serum- och urinelektrofores såväl som fria lätta kedjor i\nserum kontrolleras. Nästan alla patienter med sjukdomen har patologiska värden i\nett eller flera av dessa tester men det är betydligt vanligare att ett\npatologiskt värde beror på en harmlös monoklonal gammopati (MGUS) än\nAL-amyloidos och dessa analyser räcker inte för att ställa diagnos.\n\nDiagnos kan ställas genom biopsi av drabbat organ (om det är möjligt). Eftersom\ndetta ofta kan vara svårt, t ex biopsi från hjärtat, är det vanligare att\ndiagnosen fås genom benmärgs- och öppen fettvävsbiopsi. I en biopsi kan man\nsärskilja AL-amyloidos från annan amyloidos med immunfärgningar för kappa och\nlambda. Patologiska laboratoriet vid Akademiska sjukhuset i Uppsala är särskilt\ninriktat på amyloidosdiagnostik.\n\nEfter att sjukdomen bekräftats skall man göra en kartläggning av\nsjukdomsutbredning. Här bör man komma ihåg att ett normalt ultraljud av hjärtat\ninte utesluter hjärtengagemang utan kontroll av markörer såsom troponin T och\nNT-proBNP är än mer känsliga. MR hjärta är av värde vid vissa fall.\nSkelettscintigrafi är en värdefull metod för att skilja mellan\namyloidosengagemang i hjärtat orsakat av AL-amyloidos respektive av\ntranstyreintyp. Metoden utfaller med positivt fynd för den sistnämnda formen av\namyloidos men inte av den första.\n\nVissa internationella centra använder sig av masspektrometri för att\ndifferentiera mellan olika typer av amyloidos och metoden kommer sannolikt att\nockså införas i Sverige. Slutligen finns en unik scintigrafisk metod, SAP-scan,\nvid National Amyloidosis Centre i London med vars hjälp man kan kartlägga\nutbredning av amyloidos. Vid differentialdiagnostiska problem kan man vända sig\ndit (Länk [https://www.ucl.ac.uk/]).\n\n* Monoklonal gammopati (MGUS)\n   [https://www.internetmedicin.se/hematologi/monoklonal-gammopati-mgus] och\n   proteinuri, kardiomyopati, Polyneuropati och multipel mononeuropati\n   [https://www.internetmedicin.se/neurologi/polyneuropati-och-multipel-mononeuropati]\n   och/eller hepatomegali av annan orsak är mycket vanligare än AL-amyloidos.\n   Amyloidos påvisad vid hjärtbiopsi kan bero på så kallad senil kardiell\n   amyloidos orsakad av amyloidomvandling av transtyretin, vilken särskiljs via\n   subtypning av amyloiden.\xa0\n\n * Amyloidos sekundär till inflammatorisk sjukdom liksom amyloidos av typ AA\n   utesluts genom subtypning av amyloiden (görs främst på patologiska\n   laboratoriet Akademiska sjukhuset i Uppsala).\xa0\n\n * Amyloidos sekundär till Myelom (multipelt myelom, myelomatos)\n   [https://www.internetmedicin.se/hematologi/myelom-multipelt-myelom-myelomatos]\n   kan särskiljas genom att patienten uppfyller diagnoskriterierna för myelom.\n   (Observera att det finns patienter med AL-amyloidos som rent formellt\n   uppfyller kriteriena för myelom genom hög andel plasmaceller i benmärg. Om\n   allt annat i den kliniska bilden talar för AL-amyloidos bör patienten då\n   behandlas efter detta och inte erhålla myelomdiagnos).\n\nAll behandling vid AL-amyloidos bör skötas vid hematologiska enheter med stor\nerfarenhet av sjukdomen. Behandlingen går ut på att försöka stänga av den\nmonoklonala produktionen av lätta kedjor. Lyckas man kan den inlagrade\nAL-amyloiden långsamt lösas upp och symtomen förbättras. Eftersom det förefaller\nsom om organskada inte bara uppstår till följd av inlagrat protein utan också av\nen direkt toxisk effekt av involverad lätt kedja så är snabb effekt av\nbehandlingen eftersträvansvärd.\n\nÄnnu finns ingen behandling som kan påskynda upplösning av inlagrat protein utan\nman är enbart hänvisad till att sänka produktionen av proteinet så pass mycket\natt den underskrider den nedbrytning som långsamt pågår. Fas II studier med bl a\nmonoklonala antikroppar riktade mot den amyloida inlagringen pågår. Preliminära\ndata tyder på att de kan ha effekt vid avancerad sjukdom.\n\nEftersom sjukdomen utgår från plasmaceller har det varit naturligt att\nbehandlingar med visad effekt vid multipelt myelom också prövats vid\nAL-amyloidos. Vanligen ger man någon form av kombinationsbehandling baserad på\nbortezomib [https://www.fass.se/LIF/result?userType=0&query=bortezomib] (Velcade\n[https://www.fass.se/LIF/result?userType=0&query=Velcade]).\n\nEn randomiserad studie har visat att kombination av den monoklonala antikroppen\ndaratumumab (Darzalex), bortezomib, cyklofosfamid och kortison, Dara-VCD, ger\nmycket bra behandlingssvar och lång tids sjukdomskontroll. Denna kombination\nrekommenderas därför till alla patienter för vilka man har remissionssyftande\nambitioner med behandlingen. Behandlingsmålet är komplett hematologisk remission\nvilket innebär normalisering av serum- och urinelfores samt av fria lätta kedjor\ni serum. Om man p g a patientens ålder och/eller komorbiditer endast har\nsymtomlindring som behandlingsmål kan enbart bortezomib och kortison eller\nmelfalan [https://www.fass.se/LIF/result?userType=0&query=melfalan]\xa0(Alkeran\n[https://www.fass.se/LIF/result?userType=0&query=Alkeran]) i kombination med\nhögdos\xa0dexametason\n[https://www.fass.se/LIF/result?userType=0&query=dexametason]\xa0vara alternativ.\nTill yngre patienter som inte når komplett hematologisk remission efter 4 cykler\nDara-VCD och som inte har svår hjärtpåverkan bör högdos\xa0melfalan\n[https://www.fass.se/LIF/result?userType=0&query=melfalan]\xa0200 mg/m2\xa0i.v. följt\nav infusion av autologa stamceller, så kallad autolog stamcellstransplantation\n(ASCT) övervägas.\n\nVidare finns det Fas II-studier med positiva data för i stort sett samtliga\nläkemedel som används vid myelom, inklusive senare generationers\nproteasomhämmare och immunmodulerare (IMIDs). Det man bör tänka på om man\növerväger att använda något av dessa läkemedel främst avsedda för myelom, är att\ntolerabiliteten kan skilja sig såtillvida att en patient med AL-amyloidos oftast\ntolererar läkemedel som kan ge cytopenier väl eftersom de har en frisk benmärg.\nDäremot behöver man iaktta extra försiktighet med läkemedel som kan vara\nkardiotoxiska eller ger neuropatier. Man bör också reducera kortisondoserna till\npatienter med hjärtpåverkan. Ett intressant preparat för vilket mer data\ninväntas inom kort är\xa0venetoclax\n[https://www.fass.se/LIF/result?userType=0&query=venetoclax]\xa0(Venclyxto\n[https://www.fass.se/LIF/result?userType=0&query=Venclyxto]), en BCL2-hämmare\nmed effekt vid ett flertal hematologiska sjukdomar. När det gäller\nplasmacellssjukdomar så har lovande resultat visats vid myelom vilka uppvisar\nkromosomavvikelsen t11:14. Denna avvikelse ses hos upp till 50 % av patienterna\nmed AL amyloidos varför en större studie nu pågår. Sedan 2023 finns det i\nSverige möjlighet att behandla myelom med så kallade bispecifika antikroppar.\nDet finns även fallserier publicerade där denna typ av antikroppar har använts\nvid AL amyloidos. Preliminära resultat tyder på en mycket god effekt med\nrelativt lite biverkningar även när behandlingen givits till patienter med\navancerad sjukdom. Uppföljningstiden är dock ännu begränsad. Om sjukdomen varit\nstabil under en längre tid efter behandlingen kan patienter med avancerad\nnjurpåverkan bli accepterade för njurtransplantation. I enstaka\nnydiagnosticerade fall kan patienter med svår hjärtpåverkan bli accepterade för\nhjärttransplantation följd av systemisk behandling, främst ASCT.\n\nAL-amyloidos kan förekomma i lokaliserad form. Sjukdomen sitter då främst i\nögonlock, luft- eller urinvägarna men kan förekomma var som helst i kroppen.\n\nMan bör i dessa fall kontrollera elfores i serum och urin samt fria lätta kedjor\ni serum för att utesluta generell klonal plasmacellsjukdom. I vissa fall kan man\nbehöva genomföra en komplett utredning med benmärgs- och fettvävsbiopsi och\neventuell skelettöversikt. Om man inte finner tecken på generaliserad sjukdom\nbehandlas lokaliserad AL-amyloidos med lokal excision.\n\nRisken för utveckling av generaliserad sjukdom är mycket liten men man kan\nbehöva följa upp patienten med några års mellanrum samt vid förnyade symtom då\nlokala recidiv inte är ovanliga.\n\n2022 publicerades uppdaterade nationella riktlinjer för utredning och behandling\nav AL-amyloidos. De finns att läsa på Svensk förening för Hematologis hemsida:\n\nRiktlinjer för behandling av AL-amyloidos [https://www.sfhem.se/riktlinjer]\n\nSystemisk amyloidos är en sjukdomsgrupp där cirkulerande proteiner felveckas och\nlagras in i kroppens vävnader, vilket orsakar sjukdom. Light chain (AL)\namyloidos och transtyretin (ATTR) amyloidos är de amyloidsjukdomar som orsakar\nden absoluta majoriteten av hjärtamyloidosfallen i Sverige.\n\nHjärtamyloidos klassificeras som en hypertrofisk och restriktiv kardiomyopati, d\nv s ger en hjärtmuskelförtjockning och nedsatt eftergivlighet i myokardiet.\nSåledes leder hjärtamyloidos till hjärtsvikt med initialt bevarad systolisk\nfunktion (HFpEF) och först sent i sjukdomsförloppet nedsatt ejektionsfraktion i\nvänster kammare (HFrEF). Tillståndet är ovanligt och dess kliniska fynd liknar\ndem vid t ex hypertensiv hjärtsjukdom eller sarkomerisk Hypertrofisk\nkardiomyopati (HCM)\n[https://www.internetmedicin.se/kardiologi/hypertrofisk-kardiomyopati-hcm].\nPatienterna får därför initialt ofta fel diagnos.\n\nPrognos och behandling skiljer sig avsevärt mellan de olika sjukdomsgrupperna.\nSnabb och korrekt identifiering är därför av yttersta vikt.\n\nSe även följande behandlingsöversikter:\n\nHjärtsvikt, med bevarad systolisk funktion (HFpEF)\n\n[https://www.internetmedicin.se/kardiologi/hjartsvikt-med-bevarad-systolisk-funktion-hfpef]\n\nHjärtsvikt, kronisk – med reducerad systolisk funktion (HFrEF, HFmrEF och\nHFimpEF)\n\n[https://www.internetmedicin.se/kardiologi/hjartsvikt-kronisk-med-reducerad-systolisk-funktion-hfref-hfmref-och-hfimpef]\n\n\nPREVALENS\n\nFöljande grupper av hjärtamyloidoser har högst prevalens i Sverige:\n\n * AL-amyloidos 1/100 000, varav ca 60 % utvecklar hjärtmanifestationer.\xa0\n\n * Ärftlig transtyretinamyloidos (ATTRv) 1/100 000, högre i norra Sverige.\xa0\n\n * Wild-type transtyretinamyloidos (ATTRwt) Kallades tidigare senil systemisk\n   amyloidos. Tydligt ålders och könsberoende prevalens med manlig dominans. En\n   mindre svensk prevalensundersökning visade att minst 1:800 äldre än 70 har\n   symtomgivande ATTRwt kardiomyopati (ref Lindmark et al).\n\nAL-AMYLOIDOS\n\nAL-amyloidos [https://www.internetmedicin.se/hematologi/al-amyloidos]orsakas av\nfelveckade lätta immunoglobulinkedjor. Sjukdomen är ibland en delmanifestation\nvid myelomsjukdom eller lymfom men oftast produceras immunoglobulinet som är\nprekursorprotein till amyloiden av en plasmacellsklon som i sig inte ger\nsjukdomsmanifestationer. Sjukdomen ger sällan enbart hjärtmanifestationer, men\nhjärtsvikt är inte sällan det som för patienten till sjukvården. Andra vanliga\nkomplikationer är bl a:\n\n * Njurpåverkan med proteinuri\n\n * Polyneuropati\n\n * Hepatomegali\n\nHjärtengagemanget vid AL-amyloidos kännetecknas av snabbt progredierande\nkardiomyopati. Oftast ses hypertrofi, men i enstaka fall är restriktiviteten\ndominerande och symtomgivande hjärtsvikt kan ses i avsaknad av signifikant\nhjärtmuskelförtjockning.\n\n\nÄRFTLIG TRANSTYRETINAMYLOIDOS (ATTRV)\n\n(se PMTransthyretin-amyloidos, familjär\n[https://www.internetmedicin.se/klinisk-genetik/transthyretin-amyloidos-familjar])\n\nATTRv amyloidos är en systemsjukdom med autosomalt dominant nedärvningsmönster\nsom orsakas av varianter i transtyretingenen (TTR). Penetransen verierar och vid\nsläktanamnes kan ärftligheten vara svår att fånga. Ett hundratal\nsjukdomsorsakande varianter har hittills identifierats, varav ca 15 påvisats i\nSverige. I Sverige dominerar TTRV30M varianten som förekommer endimiskt i Norr\noch Västerbotten. ATTRv amyloidos kan förutom kardiomyopati också orsaka:\n\n * Polyneuropati\n\n * Gastrointestinalsymtom (illamående, diarré)\n\n * Ögonförändringar (glaskroppsinlagringar, glaukom)\n\n * Njurpåverkan (proteniuri och nedsatt GFR)\n\n * Karpaltunnelsyndrom\n\nHjärtmanifestationer uppträder sällan före 40 års ålder, men vid visa\nsjukdomsorskande varianter ses tidigare insjuknande. Vid hjärtpåverkan ses\nfrämst progredierande hypertrofisk kardiomyopati. Restriktivitet uppstår senare\ni förloppet. Nedsatt systolisk vänsterkammarfunktion uppstår först i sjukdomens\nslutstadier.\n\n\nWILD-TYPE TRANSTYRETINAMYLOIDOS\n\nATTRwt amyloidos eller senil systemisk amyloidos som det tidigare kallats\ndrabbar främst äldre, > 60 års ålder, och är den vanligaste formen av\nhjärtamyloidos. 90 % av patienerna är män. Amyloiden bildas också vid denna\nsjukdom från transtyretin, men utan bakomliggande genetisk defekt. Trots att\nsjukdomen kallas systemisk, och amyloid kan påvisas i hela kroppen, orsakar den\ntill skillnad från AL-amyloidos och ATTRv amyloidos huvudsakligen kardiomyopati.\nSjukdomen orsakar sällan betydande polyneuropati och njursvikt. Många patienter\ndrabbas också av karpaltunnelsyndrom, spinal stenos och senrupturer, vilka ofta\nföregår symtomgivande hjärtsvikt med flera år.\n\nHjärtmanifestationerna liknar i allt väsentligt de vid den ärftliga\ntranstyretinamyloidosen, men insjuknandeåldern är högre och progressen något\nlångsammare.\n\nHJÄRTSVIKT\n\nHjärtsviktssymtomen vid amyloid kardiomyopati är ofta mer diffusa än vid många\nandra former av hjärtsvikt. Symtom på högersvikt kan dominera och det är\nsällsynt med lungödembild. Nedsatt fysisk arbetsförmåga och muskeltrötthet kan\nvara patientens enda symtom även av långt utvecklad hjärtpåverkan.\n\nVid AL-amyloidos och kardiomyopati progredierar symtom och klinisk bild ofta\nsnabbt medan ATTR amyloidos har ett betydligt långsammare förlopp.\n\n\nARYTMIER\n\n * Bradyarytmier (både sinusknutedysfunktion och AV-block)\n\n * Förmaksflimmer\n\nArytmierna kommer inte sällan innan hjärtmuskelpåverkan blir så allvarlig att\npatienten får sviktsymtom.\n\n\nPOLYNEUROPATI\n\nVid ATTRv och AL amyloidos sker ofta utveckling av polyneuropati parallellt med\nhjärtengagemanget.\n\n* Vänsterkammarhypertrofi på grund av hypertoni eller aortastenos\n\n * Hypertrofisk kardiomyopati (HCM)\n   [https://www.internetmedicin.se/kardiologi/hypertrofisk-kardiomyopati-hcm]\n\n * Annan restriktiv kardiomyopati (sarkoidos, Hemokromatos, primär\n   [https://www.internetmedicin.se/internmedicin/hemokromatos-primar]), gäller\n   främst AL-amyloidos.\n\nAmyloid hjärtsjukdom bör alltid misstänkas vid vänsterkammarhypertrofi som\ndebuterar efter 50 års ålder och även i yngre åldrar vid samtidig:\n\n * Polyneuropati\n\n * Njursvikt med proteinuri\n\n * Karpaltunnelsyndrom\n\n * Spinal stenos\n\n * Ekokardiografiska fynd av restriktivt fyllnadsmönster också i avsaknad av\n   hypertrofi\n\n * AV-blockeringar\n\n * Lägre QRS-amplituder i EKG än väntat vid vänsterkammarhypertrofi\n\n * Svårt symtomatiskt förmaksflimmer\n\n\nANAMNES\n\n * Funktionsnivå\xa0\n\n * Sviktsymtom Dyspné? Ortopné? Ankelödem? Palpitationer? Viktuppgång?\xa0\n\n * Yrselattacker, synkope\xa0\n\n * Familjär förekomst av hjärtsvikt/polyneuropati\xa0\n\n * Känselstörningar i händer eller fötter\xa0\n\n * Spinal stenos Smärta i ben? Nedsatt gångförmåga?\xa0\n\n * Karpaltunnelsyndrom Nedsatt styrka i handen? Smärta i handen? Ses vid ATTRwt\n   eller ATTRv.\n\n\nSTATUS\n\nStatusfynden vid hjärtamyloidos är ospecifika. Ofta präglas bilden av\nhögerkammarsvikt. Lungödem är ovanligt, perfiera ödem och leverförstoring är\nvanligare.\n\n * Blodtryck\n   \n   * Vid AL amyloidos och ATTRv amyloidos ses ofta autonom dysfunktion och mer\n     eller mindre uttalad ortostatism. Blodtryck bör därför kontrolleras i både\n     sittande och stående ställning.\xa0\n\n * Neurologi\n   \n   * Tecken på polyneuropati?\n   \n   * Nedsatt perifer sensibilitet?\n   \n   * Muskelatrofi?\n\n\nLAB\n\n * Na, K, kreatinin Njurpåverkan vanligt vid AL-amyloidos.\xa0\n\n * Troponin och nT-proBNP Ofta förhöjt vid alla former av hjärtamyloidos.\n   Förhöjda värden kopplade till sämre prognos.\xa0\n\n * Serum-fria lätta kedjor / serum- och urinelektrofores Förhöjda halter av fria\n   kedjor (oftast lambda) ses ofta vid AL-amyloidos. Avvikande kappa/lambda-kvot\n   i serum har hög sensitivitet men är inte diagnostiskt. Vid misstanke om\n   amyloid kardiomyopati ska s-fria lätta kedjor samt M-komponent i urin och\n   serum analyseras (med immunofixation) som första steg i utredningen. Normala\n   s-fria lätta kedjor OCH normala elektroforeser analyserade med immunofixation\n   gör AL amyloidos väldigt osannolikt.\n\n\nEKG\n\n * Low voltage Låga QRS-amplituder ses ibland vid främst AL-amyloidos. Vid ATTR\n   amyloidos ses ofta normala QRS-amplituder trots relativt uttalade\n   hypertrofitecken vid bildundersökning.\xa0\n\n * Pseudoinfarktmönster QS-komplex anteriort som imiterar genomgången\n   hjärtinfarkt.\xa0\n\n * AV-block av varierande grad\xa0\n\n * Extremt vänsterställd elaxel LAH eller vänstersidigt skänkelblock är mycket\n   vanligt.\xa0\n\n * ST och T-förändringar Lätta ospecifika främst T-sänkningar är vanliga.\n\n\nBILDUNDERSÖKNINGAR\n\nEKOKARDIOGRAFI (UCG)\n\nFynd som talar för hjärtamyloidos:\n\n * Koncentrisk, symmetrisk vänsterkammarhypertrofi. Uttalad hypertrofi talar för\n   ATTR amyloidos.\xa0\n\n * Förtjockning av vänster- och högerkammarväggar, förmakseptum samt\n   AV-klaffar.\xa0\n\n * Perikardutgjutning\xa0\n\n * Restriktivt fyllnadsmönster (hög E/A-kvot, kort decelerationstid över\n   mitralisostiet).\xa0\n\n * Förstorade förmak\xa0\n\n * ”Sparkling echoes”\xa0\n\n * "Apical sparing" vid strainanalys av vänster kammare.\n\nMR\n\nGer ofta men långt ifrån alltid typiska fynd. Förutom ökad koncentrisk\nhypertrofi kan fläckvis eller generell LGE (late gadolinium enhancement) ses.\nVävnadskaraktärisering (T1-mapping och extracellulärvolym (ECV) bestämning) är\nvärdefullt då man oftast ser tydligt avvikande fynd.\n\n99MTCDPD-SCINTIGRAFI (SKELETTSCINTIGRAFI)\n\n99mTcDPD är en scintigrafisk markör som tidigare använts vid\nskelettskintigrafier. Metoden är mycket enkel och extremt känslig för påvisande\nav kardiomyopati vid båda typer av transtyretinamyloidos. Upptaget av DPD\ngraderas från 0 till 3. Upptag grad 0 ses hos patienter utan hjärtamyloidos och\nhos de flesta patienter med AL amyloidos. Grad 1 ses hos vissa patienter med AL\namyloidos och väldigt tidigt i sjukdomsförloppet vid ATTR amyloidos. Upptag grad\n2 och 3 ses hos majoriteten av patienter med ATTR kardiomyopati och hos ett\nfåtal patienter med AL amyloidos. Undersökningsmetoden finns idag tillgänglig i\nde flesta sjukvårdsregioner i Sverige.\n\n\nBIOPSI\n\nAmyloidosdiagnosen bör om minsta misstanke om att AL amyloidos föreligger\nverifieras med en vävnadsbiopsi. Via en biopsi kan amyloiden klassificeras och\netiologi (AL eller transtyretin) bestämmas. Prover bör skickas till erfaret\nlaboratorium. Då amyloid deponeras fläckvis, utesluter man inte sjukdom med en\nnegativ biopsi.\n\nHjärtmuskelbiopsi är inte nödvändigt, men har ofta högre känslighet för sjukdom\nän andra vävnader. För diagnosen hjärtamyloidos krävs:\n\n * Påvisande av amyloid i valfri vävnad, t ex bukfett\n\n * För diagnosen talande kliniska fynd (IVSd > 12 mm)\n\nVid ATTRwt amyloidos är bukfettsbiopsi positiv i 30-80 % av fallen, vid ATTRv\namyloidos 75-90 % och vid AL amyloidos 75-90 % av fallen.\n\nVid negativ bukfettsbiopsi kan man gå vidare med hjärtmuskelbiopsi eller biopsi\nfrån tarm, njure eller annat afficerat organ.\n\n\nGENETISK UTREDNING\n\nVid transtyretinamyloidos rekommenderas genetisk utredning för att fastställa om\ndet rör sig om ATTRwt eller ATTRv. Detta görs genom sekvensering av\ntranstyretingenen efter att diagnosen transtyretinamyloidos fastställts. Inför\nsådan testning bör genetisk vägledning hos kunnig personal erbjudas och positiva\nfynd måste följas upp.\n\n\nUTREDNINGSALGORITM\n\nETIOLOGISK BEHANDLING\n\nBehandlingen av den underliggande amyloidossjukdomen sköts av specialist på\nunderliggande sjukdomstillstånd. Vid AL-amyloidos bör snar kontakt tas med\nhematolog.\n\nEtt antal läkemedel finns tillgängliga för behandling av ATTR-amyloidos. Två\nbehandlingsprinciper har visat sig effektiva i att bromsa sjukdomsprogress vid\nbåde hjärt- och nervengagemang med liknande kliniska effekter vid amyloid\nkardiomyopati. TTR-stabilierare är perorala mediciner som motverkar\namyloidbildning genom att hålla transtyretinet i sin normala form i blodet medan\nintravenösa och subkutana TTR-sänkande läkemedel minskar produktionen av\ntranstyretin i levercellerna.\n\nTTR-stabiliserarna acoramidis\n[https://www.fass.se/LIF/result?query=acoramidis&userType=0] (Beyonttra\n[https://www.fass.se/LIF/result?query=Beyonttra&userType=0]) och tafamidis\n[https://www.fass.se/LIF/result?userType=0&query=Tafamidis] (Vyndaqel\n[https://www.fass.se/LIF/result?query=Vyndaqel&userType=0]) samt TTR-sänkaren\nvutrisiran [https://www.fass.se/LIF/result?query=vutrisiran&userType=0]\n(Amvuttra [https://www.fass.se/LIF/result?query=Amvuttra&userType=0]) (har i\nrandomiserade, placebokontrollerade, studier visat sig bromsa försämring av\nhjärtsviktsymtom och ge positiv effekt på överlevnad. Bäst effekt av\nbehandlingen sågs hos patienter med lindrigare symtom, så det är av vikt att\nbehandling sätts in innan sjukdomen progredierat för långt. Läkemedlen\nrekommenderas så främst för behandling av patienter med symtomatisk hjärtsvikt i\nNYHA-klass 1 och 2.\n\nFörskrivning ska ske vid universitetssjukhus och behandlade patienter följas upp\nvia kvalitetsregistret SweATTR. För närmare information kring SweATTR, kontakta\nAmyloidoscentrum, medicinkliniken, NUS. Se adress nedan.\n\n\nHJÄRTSVIKTSBEHANDLING\n\nPrognosvinsterna av konventionell hjärtsviktsbehandling är dåligt studerade vid\namyloid kardiomyopati. Vissa patienter upplever påtagliga biverkningar av\nmedicinering. Föreligger annan indikation för behandling eller nedsatt LVEF\nmåste medicineringen individualiseras.\n\nBETABLOCKAD\n\nOfta är patienterna p g a sin oförmåga till ökad slagvolym mycket känsliga för\nbetablockad.\n\nACE-HÄMMARE/ARB\n\nMånga av patienterna tål inte ACE-hämmare eller ARB p g a:\n\n * Autonom neuropati\n\n * Hypotension/ortostatism\n\n * Njurengagemang (vid AL-amyloidos)\n\nDIGITALIS\n\nFörsiktighet bör iakttas vid behandling med digitalispreparat då de vid\nhjärtamyloidos kan anrikas i hjärtvävnaden vilket ökar risken för biverkningar.\n\nDIURETIKA\n\nBehandling med loop-diuretika och mineralokortikoidreceptorantagonister har ofta\ngod symtomlindrande effekt.\n\nSGLT2-HÄMMARE\n\nEffekten av SGLT2-hämmare har inte studerats vid amyloid kardiomyopati, men\nanvänds ofta då symtomlindrande effekt påvisats i studier av hjärtsvikt med\nannan etiologi.\n\n\nPACEMAKER OCH ICD\n\nDet är vanligt med AV-block och andra bradyarytmier. Pacemakerimplantation bör\nutföras på frikostiga indikationer.\n\nErfarenheten av ICD-behandling är mycket begränsad. I de mindre\nbehandlingsserier på AL-amyloidos som gjorts hade behandlingen tveksam effekt.\nICD-behandling rekommenderas endast som sekundärprofylax.\n\n\nANTIKOAGULATION\n\nFörmaksflimmer är väldigt vanligt hos patienter med hjärtamyloidos. Patienter\nmed hjärtamyloidos och förmaksflimmer har mycket hög risk för kardiell\nembolisering och stroke oavsett CHA2DS2-VASc\n[https://icd.internetmedicin.se/riktlinjer/CHA2DS2-VASc]-score. Indikationen för\nantikoagulationsbehandling är ett betrakta som mycket stark.\n\nRegelbunden (årlig) uppföljning med långtids-EKG-registrering rekommenderas för\natt upptäcka intermittent bradyarytmi och förmaksflimmer. Uppföljning av\npatienter med etiologisk behandling bör ska vid eller i samråd med\nuniversitetssjukhus och registreras i SveATTR\n\nPrognosen vid hjärtamyloidos är främst beroende på underliggande\namyloidossjukdom.\n\nFaktorer associerade med sämre överlevnad är:\n\n * Förhöjda nivåer av NT-proBNP och troponin\n\n * Nedsatt njurfunktion\n\n * Nedsatt vänsterkammarfunktion\n\n * Låg funktionsnivå\n\nHistoriskt har prognosen vid AL-amyloidos med hjärtengagemang betraktats som\nmycket dålig. Bättre behandling av den underliggande plasmacellsjukdomen gör\ndock att fler patienter lever längre och att man i vissa fall ser regress av\nhjärtengagemang.\n\nVid transtyretinamyloidos är prognosen något bättre med en medianöverlevnad\nkring 3-5 år från diagnos.\n\n\nAMYLOIDOSCENTRUM\n\nMedicinmottagningen Norrlands universitetssjukhus 90185 Umeå Telefon: 090-785 39\n59\n\nAmyloidossjukdomar orsakas av extracellulära inlagringar av felveckade proteiner\nsom bildar fibriller (amyloid) som deponeras i olika vävnader och successivt\nförstör deras struktur och funktion. Kännetecknade för alla amyloidossjukdomer\när den specifika uppbyggnaden av amyloidfibrillen av beta-lakan (”beta pleated\nsheets”) som ger ett karakteristiskt utseende vid elektronmikroskopi samt den\nklassiska gröna färgbrytningen efter färgning av vävnadspreparat med kongorött\noch mikroskopisk undersökning i polariserad ljus.\n\nMånga proteiner kan bilda amyloid, och även om flertalet inte ger upphov till\nsjukdom finns det kliniskt betydelsefulla och ofta fatala sjukdomar som beror på\namyloidbildning. Man brukar dela upp amyloidossjukdomarna i två huvudgrupper:\nlokaliserade och generaliserade (systemiska), varav den senare gruppen i sin tur\nlämpligen indelas i två former: sporadisk och familjär (ärftlig).\n\nTransthyretin-amyloidos (TTR-amyloidos) är den vanligaste formen av ärftlig\namyloidos, och går även under namnen familjär amyloidos med polyneuropati (FAP)\neller ”Skelleftesjukan” eller familjär amyloidos med kardiomyopati (FAC),\nberoende på mutation och presentation.\n\nDet finns ett flertal andra proteiner som orsakar familjära former av amyloidos,\nsåsom apolipoprotein AI och AII, gelsolin, fibrinogen, lysozym och cystatin C.\nMen av dessa är enbart enstaka fall av gelsolin-amyloidos diagnostiserade i\nSverige, då hos individer med finsk bakgrund. Fibrinogen-amyloidos är\nrapporterad från Europa och USA som en vanlig orsak till hereditär njursvikt,\nmen inget fall finns rapporterat från Sverige. En familj med lysozym-amyloidos i\nUSA skall ha svenskt ursprung men ingen med denna form av sjukdom har beskrivits\ni Sverige.\n\nTTR-amyloidos beskrevs först av Andrade i Portugal och senare rapporterades\nfamiljer från Sverige och Japan. De första rapporterna rörande TTR-amyloidos\nbetonade perifer neuropati, vilket ledde till begreppet familjär amyloidos med\npolyneuropati (FAP). Även om perifer neuropati oftast är ett första symtom på\nsjukdomen så är TTR-amyloidos en generaliserad sjukdom med symtom från flera\nolika organsystem.\n\n\nPATOGENES\n\nTTR är ett transportprotein som syntetiseras till över 90 % i levern men det\nproduceras även i retina, pankreas och plexus choriodeus. TTR binder tyroxin och\nretinolbindande protein (RBP) i plasma och cerebrospinalvätska och deltar på så\nsätt i transporten av vitamin A.\n\nTTR förekommer i plasma som en homotetramer, det vill säga att fyra identiska\nTTR-molekyler är bundna till varandra. TTR-genen är belägen på kromosom 18, och\növer 100 olika enkel- eller dubbel-punktmutationer är beskrivna. Mutationerna\närvs autosomalt dominant och den vanligaste mutationen i Sverige är metionin i\nstället för valin i position 50 i TTR-genen (position 30 i det mogna proteinet,\nTTR V30M).\n\nAminosyrasubstitutioner leder till konformationsförändringar av TTR som leder\ntill svagare subenhetinteraktioner i den tetrameriska strukturen, vilket i sin\ntur leder till splittring av tetrameren till monomerer. Efter formförändringar\nbildar dessa amyloidfibriller. Dissociationen till monomerer är det första\nsteget i utvecklingen av amyloidos, med vad som initierar amyloidbildning är\nännu inte fullständigt klarlagt.\n\nTTR-amyloidos är, som alla amyloidossjukdomar, ett tillstånd som först bryter ut\nefter puberteten, oftast efter 20-års ålder. I Sverige debuterar TTR\nV30M-amyloidos oftast i medelåldern, även om sjukdomen kan debutera i\n20-årsåldern. Denna relativt sena debut kan antingen bero på att de mekanismer\nsom främjar amyloidbildning blir mer aktiva, eller att förebyggande faktorer\nblir mindre effektiva med ökande ålder.\n\nDå även icke-muterat TTR ofta bildar amyloidinlagringar hos äldre individer\n(senil systemisk amyloidos) är det uppenbart att en TTR-mutation inte är en\navgörande faktor för amyloidfibrillbildning.\n\n\nEPIDEMIOLOGI\n\nÄrftlig TTR-amyloidos drabbar 1/100 000 personer, men antalet varierar mellan\nolika länder. I Sverige är Västerbotten och Norrbotten endemiska områden, med en\nförekomst av TTR V30M på 1/1000, liknande den man observerar i de endemiska\nområdena i Portugal och Japan.\n\nMutationer med dominerande hjärtmanifestationer har identifierats i Sverige i\nbland annat Dalarna och Jämtland, samt hos individer med utländsk härkomst genom\nexempelvis TTR V122I-mutationen, som förekommer i 3,5 % av den afro-amerikanska\nbefolkningen.\n\nDet är viktigt att man görs uppmärksam på att ca 50 % av patienter som\ndiagnostiseras med ärftlig TTR-amyloidos inte har någon känd hereditet. Avsaknad\nav sjukdom i släkten utesluter alltså inte förekomst av ärftlig TTR-amyloidos.\nDetta illustreras av att manifestationen av TTR V30M-mutationen varierar. I\nSverige beräknas 5-10 % av bärarna utveckla sjukdom. Detta förhållande tyder på\natt det finns epigenetiska och/eller miljöfaktorer som påverkar\nsjukdomsuppkomst.\n\nI Sverige är sjukdomsdebut vanligast vid ca 60 års ålder, medan den i Portugal\noch Japan sker vid ca 35-40 års ålder. Man brukar tala om ”early onset” (under\n50 år) och ”late onset” (över 50 år). ”Late onset”-formen är vanligast i\nSverige.\n\nVid ärftlig TTR-amyloidos finns symtom från flera olika organsystem. Hos den\nvanligaste svenska mutationen dominerar neuropati, men även symtom från hjärta,\nmag-tarmkanal, ögon och njurar är vanliga.\n\n\nNEUROLOGISKA SYMTOM\n\nEn symmetrisk, axonal, perifer, sensorisk-motorisk neuropati är den vanligaste\nmanifestationen. Neuropatin är längd-beroende, på så sätt att den börjar distalt\ni extremiteterna (”glove and sock neuropathy”) för att successivt progrediera\ncentralt. Typisk är förekomst av smärta. EMG-förändringar kan komma relativt\nsent efter symtomdebut, och patienterna har i tidiga stadier oftast enbart\nstörningar av temperaturkänslan. Det finns dock varierande presentation, och\nneuropatin behöver inte vara symmetrisk.\n\nAutonoma symtom, såsom ortostatisk hypotension med yrsel och sfinkterstörningar\nmed urin- och avföringsinkontinens, samt gastropares (fördröjd magsäckstömning)\nförekommer ofta sent i förloppet. Impotens kan förekomma tidigt i\nsjukdomsförloppet.\n\nDet finns i Sverige mutationer som har symtom från CNS i form av kognitiva\nstörningar, TIA-liknande attacker och epilepsianfall. Dessa patienter har ofta\nockså ögonsymtom (se nedan). Diagnosen kan ställas med MR-hjärna, där man ser\nkontrastuppladdning i meningerna.\n\n\nHJÄRTSYMTOM\n\nDet typiska fyndet vid ultraljudsundersökning av hjärtat är en hypertrofisk,\nrestriktiv kardiomyopati. Emellertid är det ofta svårt att påvisa\nrestriktivitet, då patienterna utvecklar en amyloid ”hypertrofi”, som med\nekokardiografisk teknik inte kan skiljas från hypertrof kardiomyopati beroende\npå sarkomerabnormitet. Detta är tyvärr ett vanligt förekommande problem.\nKarakteristisk för amyloid kardiomyopati är att patienterna tål betablockad\ndåligt.\n\nHjärtarytmier är vanligt förekommande och kan ge upphov till yrsel samt synkope.\nDå patienterna även ofta har en autonom neuropati med bristfällig\nsympatikusaktivitet är de mer utsatta för allvarliga arytmier, varför\npacemakerbehandling är indicerad även vid mindre uttalade arytmier.\n\n\nGASTROINTESTINALA SYMTOM\n\nVanligt är förstoppning som efter några år ändrar karaktär till omväxlande\nförstoppning och diarré. Så småningom lider patienter av kontinuerlig diarré,\nofta komplicerad av inkontinens. Även illamående och kräkningar är vanliga, och\nsjukdomen karakteriseras av en kontinuerlig viktnedgång. I slutstadiet av\nsjukdomen är patienterna svårt kakektiska.\n\nDet är inte ovanligt att första symtom på sjukdomen är ofrivillig viktnedgång.\nPatienternas gastrointestinala symtom beror sannolikt på motilitetsstörning i\ngastrointestinalkanalen.\n\n\nURINVÄGS- OCH NJURPÅVERKAN\n\nOfullständig tömning av urinblåsan är en vanlig komplikation, som ökar risken\nför urinvägsinfektioner, vilka kan påverka njurfunktionen. Dessutom kan\nnjurfunktionen påverkas av amyloidinlagringar i njuren, som leder till\nproteinuri.\n\nNjursvikt vid TTR-amyloidos är ovanlig, men förekommer.\n\n\nÖGONSYMTOM\n\nGlaskroppsgrumlingar är ett karakteristiskt symtom vid TTR-amyloidos, och fynden\när nästan patognomona för TTR-amyloidos.\n\nPatienterna kan även utveckla glaukom och utfällningar av amyloid i främre\nkammare som kan ses som ”dust” (damm) på insidan av linsen.\n\nVidare kan patienterna utveckla ”kugghjulspupill” beroende på autonom neuropati,\nvilket ger minskad tårproduktion och torra ögon.\n\n\nKARPALTUNNELSYNDROM\n\nVid TTR-amyloidos är karpaltunnelsyndrom, ofta bilateralt, vanligt förekommande.\nDet är inte ovanligt att tillståndet debuterar åratal innan andra symtom på\namyloidossjukdom. Komplikationen är också vanlig hos patienter med senil\nsystemisk amyloidos.\n\n\nÖVRIGA ORGANSYMTOM\n\nTTR-amyloidos är en systemisk sjukdom, varför många organ kan vara påverkade.\nBinjurebarksinsufficiens med ortostatism, illamående och kräkningar förväxlas\nlätt med gastrointestinala och autonoma symtom på TTR-amyloidos. Även spinal\nstenos kan orsakas av amyloidinlagringar.\n\nFör att få diagnosen familjär TTR-amyloidos krävs både påvisande av amyloid i\nbiopsimaterial och förekomst av en mutation i TTR-genen. Enbart positivt\ngenetiskt test räcker inte, då förekomsten av TTR-mutationen V30M är utbredd -\nåtminstone i norra Sverige - och penetransen är låg.\n\nLämplig lokal för vävnadsprov är subkutan bukfettvävnad där man lätt kan ta\nstörre biopsi för typbestämning av amyloidsjukdomen. Om bukfettvävnadsbiopsi av\nnågot skäl är olämplig eller utfaller negativ trots kvarstående\namyloidosmisstanke kan prov från GI-trakten eller nerv företas. Vid negativ\nbiopsi och kvarvarande misstanke, bör man upprepa biopsitagning och överväga\nbiopsi från t ex hjärtat om patienten har en misstänkt amyloid hjärtsjukdom.\n\nFör att säkerställa förekomst av en TTR-amyloidos bör man även karakterisera det\namyloidogena proteinet i biopsin. Utan karaktäristik av amyloidinlagringen kan\nman feldiagnostisera patienter med TTR-amyloidos som AL-amyloidos med allvarliga\nkonsekvenser för patienten.\n\nUndersökningen med 99mTcDPD-scintigrafi, även känd som skelettscintigrafi, har\nförmågan att detektera inlagringar av amyloid i hjärtat. Denna metod kan vara\nanvändbar för att fastställa diagnosen TTR-amyloidos. Dock är det viktigt att\nockså utesluta AL-amyloidos som en möjlig diagnos. Detta kan göras genom att\nanalysera serumprover för att bestämma nivåerna av fria lätta kedjor och genom\natt utföra urinanalyser för att detektera eventuell förekomst av M-komponent.\n\nLevertransplantation har länge varit det enda behandlingsalternativet för\nTTR-amyloidos. Då TTR till största delen produceras i levern kan\nlevertransplantation göra att det felveckade proteinet minskas med minst 95 %.\nLevertransplantation bör erbjudas de patienter som är i ett tidigt stadium i sin\nsjukdom och utan hjärtengagemang.\n\nTafamidis [https://www.fass.se/LIF/result?userType=0&query=Tafamidis] (Vyndaqel\n[https://www.fass.se/LIF/result?userType=0&query=Vyndaqel]), 20 mg 1 gång/dag,\när ett godkänt läkemedel för behandling av TTR-amyloidos i stadium 1 och utan\nhjärtengagemang. Verkningsmekanismen för detta läkemedel är att stabilisera\nTTR-tetrameren.\n\nDiflunisal [https://www.fass.se/LIF/result?userType=0&query=Diflunisal] (Donobid\n[https://www.fass.se/LIF/result?userType=0&query=Donobid]) är ett gammalt\nantiinflammatoriskt läkemedel som också har stabiliserande effekt på\nTTR-tetrameren och som nu i en stor multinationell studie har visats ha en bra\neffekt i alla stadier av sjukdomen och även hos patienter med hjärtengagemang.\nFör detta läkemedel krävs licens då det är avregistrerat från marknaden.\n\nInotersen [https://www.fass.se/LIF/result?userType=0&query=Inotersen] (Tegsedi\n[https://www.fass.se/LIF/result?userType=0&query=Tegsedi]), 284 mg 1 gång/vecka\nsom subkutan injektion, nytt godkänt läkemdel för behandling av TTR-amyloidos\nmed polyneuropati i stadium 1 eller 2. Detta läkemedel är en\nantisensoligonukleotid som hämmar produktionen av TTR.\n\nPatisiran [https://www.fass.se/LIF/result?userType=0&query=Patisiran] (Onpattro\n[https://www.fass.se/LIF/result?userType=0&query=Onpattro]), 300 μg/kg\nkroppsvikt som administreras intravenöst var tredje vecka, nytt godkänt läkemdel\nför behandling av TTR-amyloidos med polyneuropati i stadium 1 eller 2. Detta\nläkemedel är en dubbelsträngad liten interfererande ribonukleinsyra som hämmar\nproduktionen av TTR.\n\nTafamidis [https://www.fass.se/LIF/result?userType=0&query=Tafamidis] (Vyndaqel\n[https://www.fass.se/LIF/result?userType=0&query=Vyndaqel]), 61 mg 1 gång/dag,\när ett läkemedel som blivit nyligen godkänt för behandling av ATTR-amyloidos med\nkardiomyopata (ATTR-CM). Verkningsmekanismen för detta läkemedel är att\nstabilisera TTR-tetrameren.\n\nVutrisiran [https://www.fass.se/LIF/result?query=Vutrisiran&userType=0]\n(Amvuttra [https://www.fass.se/LIF/product?userType=0&nplId=20210913000064]), 25\nmg 1 gång/3 månader som subkutan injektion, nytt godkänt läkemedel för\nbehandling av TTR-amyloidos med polyneuropati i stadium 1 eller 2, Detta\nläkemedel är en dubbelsträngad liten interfererande ribonukleinsyra som hämmar\nproduktionen av TTR.\n\nAlla patienter som står på behandling som hämmar produktion av TTR skall ges\nvitamin A tillskott 2500 IE-3000 IE dagligen.\n\nSymtomatisk behandling för olika symtom måste tillgripas för att lindra. Vid\nneuropatisk smärta kan gabapentin\n[https://www.fass.se/LIF/result?userType=0&query=gabapentin] (Neurontin\n[https://www.fass.se/LIF/result?userType=0&query=Neurontin]) vara ett bra val.\nVid fördröjd magsäckstömning kan lågdos peroral (50 mg x 3) erytromycin\n[https://www.fass.se/LIF/result?userType=0&query=erytromycin] (Ery-max\n[https://www.fass.se/LIF/result?userType=0&query=Ery-max]) användas försöksvis\nunder en period, i svåra fall sker samverkan med gastroenterolog. Förstoppning\nkan behandlas med olika laxeringsmedel men man skall vara medveten om att under\nsjukdomen kan förstoppning övergå i diarréer. Dessa kan bero på bakteriell\növerväxt och behandlas med regelbundna antibiotikakurer i lågdos.\n\n* Kliniska kontroller med 6-12 månaders intervall (tätare initialt).\n\n * Ekokardiogram, varje till vartannat år för patienter med kardiomyopati.\n\n * Årligt EKG, ev Holter-EKG, speciellt vid symtom som kan inge misstanke om\n   arytmi.\n\n * För anlagsbärare med symtom rekommenderas upprepade bukfettbiopsier med 6-12\n   månaders intervall tills påvisad amyloidinlagring. Hjärtbiopsi bör övervägas\n   vid negativ bukfettbiopsi om ekokardiografi inger misstanke om amyloid\n   kardiomyopati.\n\nATTR-amyloidos är en mycket allvarlig sjukdom med en medelöverlevnad från\ndiagnos på 10 år. Med levertransplantation och medicinsk behandling har\nförhållandena avsevärt förbättrats men sjukdomen är fortfarande allvarlig och\nbehöver livslång behandling och monitorering.'
"""
läkemedel = scrape_clean_text('https://fass.se/health/product/20240110000048/smpc') + scrape_clean_text('https://fass.se/health/product/20190109000057/smpc')
quiz = create_quiz(article, läkemedel=läkemedel, questions=20)
print(quiz)

Här följer ett **20‑frågors flervalsquiz** som testar förståelsen av **behandling av amyloid kardiomyopati**, med fokus på allmän behandling och 1–2 frågor om specifika läkemedel (t.ex. tafamidis och akoramidis).  
Varje fråga har fyra svarsalternativ (A–D) och en pedagogisk motivering.

---

### **Quiz: Behandling av amyloid kardiomyopati**

---

#### **1.**
En 72‑årig man med restriktiv kardiomyopati och proteinuri diagnostiseras med AL‑amyloidos. Vilken är den huvudsakliga behandlingsprincipen vid denna sjukdom?
A. Öka nedbrytningen av inlagrat amyloidprotein  
B. Hämma bildningen av amyloidfibriller i hjärtat  
C. Stoppa produktionen av monoklonala lätta kedjor  
D. Ge immunosuppressiv behandling mot autoimmunitet  

**Rätt svar:** C  
**Motivering:** AL‑amyloidos orsakas av plasmaceller som producerar felveckade lätta kedjor. Behandlingen syftar till att stoppa denna produktion, t.ex. med bortezomib‑baserade regimer.

---

#### **2.**
Vilken kombination rekommenderas som förstahan

In [2]:
import pandas as pd

# Define the strings to match in column 4
links = ["https://internetmedicin.se/kardiologi/hjartamyloidos-amyloid-kardiomyopati", "https://internetmedicin.se/hematologi/al-amyloidos", "https://internetmedicin.se/klinisk-genetik/transthyretin-amyloidos-familjar"]

df = pd.read_csv("treatments.csv")

mask = df.iloc[:, 3].astype(str).isin(links)
filtered = df[mask]
filtered = filtered[~filtered.iloc[:, 4].isin(["Referenser", "ICD-10"])]
articles = filtered.iloc[:, 6]
articles
#articles_str = "\n\n".join(articles.dropna().astype(str))
#articles_str

561      Amyloidos är ett begrepp som används för en he...
562      En stor svårighet vid diagnostik av AL-amyloid...
563      Eftersom flera olika sjukdomar kan leda fram t...
564      * Monoklonal gammopati (MGUS)\n   [https://www...
565      All behandling vid AL-amyloidos bör skötas vid...
566      AL-amyloidos kan förekomma i lokaliserad form....
773      Systemisk amyloidos är en sjukdomsgrupp där ci...
774      AL-AMYLOIDOS\n\nAL-amyloidos [https://www.inte...
775      HJÄRTSVIKT\n\nHjärtsviktssymtomen vid amyloid ...
776      * Vänsterkammarhypertrofi på grund av hyperton...
777      Amyloid hjärtsjukdom bör alltid misstänkas vid...
778      ETIOLOGISK BEHANDLING\n\nBehandlingen av den u...
779      Regelbunden (årlig) uppföljning med långtids-E...
780      Prognosen vid hjärtamyloidos är främst beroend...
10943    Amyloidossjukdomar orsakas av extracellulära i...
10944    I Sverige är sjukdomsdebut vanligast vid ca 60...
10945    För att få diagnosen familjär TTR-amyloidos kr.

In [ ]:
#TODO quiz self eval

def check_quiz(quesiton, answer):
    prompt_template = """
    Roll: expert på medicin och läkemedel
    Uppgift: 
    1. Granska följande fråga och svar från ett quiz. 
    2. Bedöm om svaret är korrekt baserat på den givna frågan.
    3. Bedöm om motiveringen är korrekt eller måste förtydligas.
    4. Bedöm om man kan lägga till något till motiveringen som förklarar varför de andra alternativen är felaktiga.

    Input:
    Fråga: {question}
    Svar: {answer}
    Motivering: {justification}
    """

In [5]:
# Build nested JSON + Markdown from treatments.csv (using specified column indices)
import json
section_col = df.columns[3]      # section headings (column 3)
subsection_col = df.columns[4]   # subsection headings (column 4)
text_col = df.columns[6]         # textual information (column 6)

data = {}

for _, row in df[[section_col, subsection_col, text_col]].fillna("").iterrows():
    section = str(row[section_col]).strip() or "Untitled Section"
    subsection = str(row[subsection_col]).strip() or "Untitled Subsection"
    text = str(row[text_col]).strip()
    if not text:
        continue
    data.setdefault(section, {}).setdefault(subsection, []).append(text)

# Write JSON
with open("treatments.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

# Write Markdown
lines = []
for section, subsections in data.items():
    lines.append(f"# {section}")
    for subsection, texts in subsections.items():
        lines.append(f"## {subsection}")
        lines.extend(texts)
        lines.append("")  # blank line between subsections
    lines.append("")      # blank line between sections

with open("treatments.md", "w", encoding="utf-8") as f:
    f.write("\n".join(lines).strip() + "\n")